In [1]:
from pref_deal import *
from multiperms import multiperm2index
import numpy as np
import pandas as pd
import re

Возьмём какую-нибудь рандомную сдачу из архива:

In [2]:
df = pd.read_csv('../YellowArchive/data_406/406184500_406185000/yellow_deals.csv')
list(df.iloc[1][['north', 'east', 'south', 'west']])

['♥9♦9', '♠9♣QJ97♦AJ7♥A8', '♠Q107♦KQ♥KQJ107', '♠AKJ8♣AK108♦108']

Построим матрицу по этой сдаче:

In [11]:
%%time
hand_strings = ['♠9♣QJ97♦AJ7♥A8', '♠Q107♦KQ♥KQJ107', '♠AKJ8♣AK108♦108']
deal_matrix = PrefDealMatrix.from_strings(hand_strings, trump=2)
deal_matrix

CPU times: user 281 µs, sys: 83 µs, total: 364 µs
Wall time: 425 µs


♠9 ♧QJ97 ♦AJ7 ♥A8 
♠QT7 ♧- ♦KQ ♥KQJT7 
♠AKJ8 ♧AKT8 ♦T8 ♥-

In [12]:
%%time
deal_hash = hash(deal_matrix)
print(deal_hash)

11502728032044657
CPU times: user 2.12 ms, sys: 0 ns, total: 2.12 ms
Wall time: 1.86 ms


In [14]:
deal_matrix.trump

2

Построение матрицы и вычисление хеша занимает порядка миллисекунд, что много. Что там с восстановлением из хеша?

In [15]:
%%time
deal_from_hash = PrefDealMatrix.from_hash(deal_hash)
deal_from_hash

CPU times: user 2.56 ms, sys: 1.18 ms, total: 3.74 ms
Wall time: 3.04 ms


♠9 ♧QJ97 ♦AJ8 ♥A9 
♠QT7 ♧- ♦KQ ♥KQJT8 
♠AKJ8 ♧AKT8 ♦T9 ♥-

Тоже довольно медленно, восстанавливается сдвинутая (уплотнённая) в сторону туза матрица.

Сделаем ход:

In [25]:
deal_matrix.do_move(np.array([13, 28, 15], dtype=np.uint8))
deal_matrix

♠9 ♧J97 ♦AJ7 ♥A8 
♠QT7 ♧- ♦KQ ♥KQT7 
♠AKJ8 ♧KT8 ♦T8 ♥-

In [26]:
%%time
deal_hash_9 = hash(deal_matrix)
print(deal_hash_9)

84963744407776958
CPU times: user 1.05 ms, sys: 634 µs, total: 1.69 ms
Wall time: 1.6 ms


Упс, при восстановлении из хеша козырем стала пика:

In [28]:
%%time
deal_9_from_hash = PrefDealMatrix.from_hash(deal_hash_9)
deal_9_from_hash

CPU times: user 3.82 ms, sys: 0 ns, total: 3.82 ms
Wall time: 3.03 ms


♤9 ♣KJ9 ♦AJ8 ♥AT 
♤QT7 ♣- ♦KQ ♥KQJ9 
♤AKJ8 ♣AQT ♦T9 ♥-

In [16]:
deal_from_hash.trump

2

In [17]:
hash(deal_from_hash)

11502728032044657

In [38]:
max_hash = 0
fields = ['east', 'south', 'west', 'north']

for i in range(24):
    for n_fields in [3, 4]:
        trump = np.random.randint(17 - 4 * n_fields)
        deal = PrefDealMatrix.from_strings(list(df.iloc[i][fields[:n_fields]]), trump=trump)
        deal_hash = hash(deal)
        deal_from_hash = PrefDealMatrix.from_hash(deal_hash)
        if deal != deal_from_hash:
            # print(deal.card_matrix)
            print(deal)
            print(deal_hash)
            print(deal_from_hash)
        max_hash = max(max_hash, deal_hash)
            # print(widow, order, hash(deal))
            # tigtened_deal = PrefDealMatrix(deal.get_tightened_matrix(), order, widow)
            # print("Tightened deal:")
            # print(tigtened_deal)
            # print()
            # tight_hash = hash(tigtened_deal)
            # max_tight_hash = max(max_tight_hash, tight_hash)
            #if tight_hash != deal_hash:
            #    print(widow, order, deal_hash, tight_hash)
            #    print(deal)
    # print()
print(f"{max_hash:e}")# , f"{max_hash_widow:e}", f"{max_tight_hash:e}")

5.074758e+18


Тут первая рука играет 7 треф, поэтому переставить надо всего лишь первые две строки матрицы:

In [18]:
deal_7_clubs = PrefDealMatrix.from_strings(['♠7♣KJ7♦AQJ♥A97', '♠K♣Q9♦K97♥KJ108', '♠AQ98♣A108♦108♥Q'],
                                           trump=4)
deal_7_clubs

♠7 ♣KJ7 ♦AQJ ♡A97 
♠K ♣Q9 ♦K97 ♡KJT8 
♠AQ98 ♣AT8 ♦T8 ♡Q

In [19]:
deal_7_clubs.card_matrix

array([[  1,  64, 166],
       [ 81,  36, 138],
       [176,  69,  10],
       [133,  90,  32]], dtype=uint8)

In [20]:
PrefDealMatrix.from_hash(hash(deal_7_clubs))

♠9 ♣KJ7 ♦AQJ ♡A97 
♠K ♣Q9 ♦K97 ♡KJT8 
♠AQJT ♣AT8 ♦T8 ♡Q

In [5]:
mp = suit_codes2multiperm(deal_7_clubs.card_matrix[0], 'little', True)
mp

array([3, 2, 3, 4, 4, 3, 3, 1], dtype=int16)

In [6]:
multiperm2suit_codes(mp - 1, 'little', 4)

array([128,  64,  36,   8], dtype=uint8)

In [37]:
np.hstack((deal_7_clubs.contingency_table(), np.array([2, 0, 0, 0])[:,None]))

array([[1, 1, 4, 2],
       [3, 2, 3, 0],
       [3, 3, 2, 0],
       [3, 4, 1, 0]])

In [7]:
%%time
index = hash(deal_7_clubs)

[0 2 2 3 3 2 1 2]
840 [1 1 4 2]
53
[0 2 1 2 0 1 0 2]
560 [3 2 3 0]
170
[1 2 1 2 0 0 1 0]
560 [3 3 2 0]
407
[0 1 0 1 1 2 1 0]
280 [3 4 1 0]
42
CPU times: user 4.14 ms, sys: 247 µs, total: 4.39 ms
Wall time: 3.73 ms


In [8]:
deal_from_index = PrefDealMatrix.from_hash(index, 'big')
deal_from_index

53
840 [1 1 4 2]
[0 2 2 3 3 2 1 2]
170
560 [3 2 3 0]
[0 2 1 2 0 1 0 2]
407
560 [3 3 2 0]
[1 2 1 2 0 0 1 0]
42
280 [3 4 1 0]
[0 1 0 1 1 2 1 0]


♠7 ♣KJ7 ♦AQJ ♥A97 
♠K ♣Q9 ♦K97 ♥KJT8 
♠AQ98 ♣AT8 ♦T8 ♥Q

In [9]:
deal_from_index.card_matrix

array([[128,   2, 101],
       [138,  36,  81],
       [ 13, 162,  80],
       [161,  90,   4]], dtype=uint8)

In [10]:
deal_7_clubs.card_matrix

array([[128,   2, 101],
       [138,  36,  81],
       [ 13, 162,  80],
       [161,  90,   4]], dtype=uint8)

In [9]:
deal_7_clubs.tighten_cards()
deal_7_clubs

♠9 ♣KJ7 ♦AQJ ♥A97 
♠K ♣Q9 ♦K97 ♥KJT8 
♠AQJT ♣AT8 ♦T8 ♥Q

In [16]:
print(f"{hash(deal_7_clubs):e}")

6.744097e+15


In [12]:
%%time
widow = True
for i in range(4):
    mp = suit_codes2multiperm(deal_6_diamonds.card_matrix[i], 'big', widow)
    ns = [np.sum(mp == i) for i in range(1, 4)]
    if widow:
        ns = np.append(ns, np.sum(mp == deal_7_clubs.card_matrix.shape[1] + 1))
    print(ns, mp)
    print(multiperm2index(mp[mp > 0] - 1, ns))

NameError: name 'deal_6_diamonds' is not defined

In [5]:
deal_7_clubs.card_matrix

array([[ 32,   2,  29],
       [138,  36,  81],
       [ 13, 162,  80],
       [161,  90,   4]], dtype=uint8)

In [4]:
deal_7_clubs.contingency_table()

array([[1, 1, 4],
       [3, 2, 3],
       [3, 3, 2],
       [3, 4, 1]], dtype=uint8)

Перестановка имеет вид $(1, 0, 2, 3)$:

In [5]:
deal_7_clubs.contingency_table()[[1, 0, 2, 3]]

array([[3, 2, 3],
       [1, 1, 4],
       [3, 3, 2],
       [3, 4, 1]], dtype=uint8)

In [8]:
for i in range(4):
    print(suit_codes2multiperm(deal_7_clubs.card_matrix[i], True))

[0 2 2 3 3 2 1 2]
[0 2 1 2 0 1 0 2]
[1 2 1 2 0 0 1 0]
[0 1 0 1 1 2 1 0]


In [17]:
np.dot(np.arange(3, dtype=np.int8), np.unpackbits(deal_7_clubs.card_matrix[1], bitorder='little').reshape((3, -1)))

array([0, 2, 1, 2, 0, 1, 0, 2], dtype=int16)

In [24]:
np.unpackbits(np.invert(deal_7_clubs.card_matrix[0].sum(dtype=np.uint8)), bitorder='little')

array([0, 0, 0, 1, 1, 0, 0, 0], dtype=uint8)

А вот где третья рука играет 6 бубей:

In [6]:
deal_6_diamonds = PrefDealMatrix.from_strings(['♠QJ1087♣A10♦9♥J10', '♠K♣J8♦AKQ♥A987', '♠A9♣KQ97♦J1087'],
                                              bit_order='big', widow=True)
deal_6_diamonds

♠QJT87 ♣AT ♦9 ♥JT 
♠K ♣J8 ♦AKQ ♥A987 
♠A9 ♣KQ97 ♦JT87 ♥-

In [3]:
hash(deal_6_diamonds)

65570378535478109

In [4]:
deal_6_diamonds.contingency_table()

array([[5, 1, 2],
       [2, 2, 4],
       [1, 3, 4],
       [2, 4, 0]], dtype=uint8)

Тут потребуется перестановка $(2, 3, 1, 0)$:

In [5]:
deal_6_diamonds.contingency_table()[[2, 3, 1, 0]]

array([[1, 3, 4],
       [2, 4, 0],
       [2, 2, 4],
       [5, 1, 2]], dtype=uint8)

In [6]:
for i in range(24):
    for widow in [False, True]:
        for order in ['big', 'little']:
            deal = PrefDealMatrix.from_strings(list(df.iloc[i][['east', 'south', 'west']]), order, widow)
            # print(deal.card_matrix)
            for j in range(4):
                mp = suit_codes2multiperm(deal.card_matrix[j], order, widow)
                suit_codes = multiperm2suit_codes(mp - 1, order, 4 if widow else 3)
                if not np.array_equal(deal.card_matrix[j], suit_codes[:3]):
                    print(i, j, widow, order)
                    print(suit_codes)

In [7]:
for i in range(4):
    mp_big = suit_codes2multiperm(deal_6_diamonds.card_matrix[i], 'big', True)
    mat_big = multiperm2suit_codes(mp_big - 1, 'big')
    print(deal_6_diamonds.card_matrix[i])
    print("Big:")
    print(mp_big)
    print(mat_big)
    
    mp_little = suit_codes2multiperm(deal_6_diamonds.card_matrix[i], 'little', True)
    mat_little = multiperm2suit_codes(mp_little - 1, 'little')
    print("Little:")
    print(mp_little)
    print(mat_little)
    print()

[220   2  33]
Big:
[1 1 3 1 1 1 2 3]
[220   2  33]
Little:
[3 2 1 1 1 3 1 1]
[220   2  33]

[ 17  72 166]
Big:
[3 2 3 1 2 3 3 1]
[ 17  72 166]
Little:
[1 3 3 2 1 3 2 3]
[ 17  72 166]

[ 32   7 216]
Big:
[3 3 1 3 3 2 2 2]
[ 32   7 216]
Little:
[2 2 2 3 3 1 3 3]
[ 32   7 216]

[ 24 225   0]
Big:
[2 2 2 1 1 4 4 2]
[ 24 225   0   6]
Little:
[2 4 4 1 1 2 2 2]
[ 24 225   0   6]



In [10]:
suit_bits = np.unpackbits(deal_6_diamonds.card_matrix[3]).reshape((3, -1))
non_zero_bits = suit_bits.sum(axis=0, dtype=np.uint8) >0
suit_bits, suit_bits[:, non_zero_bits], 1 - non_zero_bits

(array([[0, 0, 0, 1, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8),
 array([[0, 1, 1, 0, 0, 0],
        [1, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0]], dtype=uint8),
 array([0, 1, 1, 0, 0, 0, 0, 0]))

In [21]:
suit_bits = np.unpackbits(deal_7_clubs.card_matrix[0], bitorder='big').reshape((3, -1))
non_zero_bits = suit_bits.sum(axis=0, dtype=np.uint8) >0
suit_bits, suit_bits[:, non_zero_bits]

(array([[1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 1, 0, 0, 1, 0, 1]], dtype=uint8),
 array([[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 1, 1, 0, 1]], dtype=uint8))

In [33]:
bits = np.unpackbits(np.array(128, dtype=np.uint8), bitorder='big')
np.packbits(bits[:-3], bitorder='big')

array([128], dtype=uint8)

In [31]:
bits

array([1, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [20]:
np.packbits(suit_bits[:, non_zero_bits], axis=1, bitorder='big').ravel()

array([  4,  64, 184], dtype=uint8)

In [3]:
hand_list = ['♠KQJ9♣J♦Q♥AJ107', '♠8♣AKQ10♦KJ7♥K8', '♠A107♣987♦A10♥Q9']
mizere_list = ['♠98♣QJ7♦QJ87♥8', '♠AQ7♣A9♦A9♥Q97', '♠KJ10♣K108♦K10♥KJ']
missing_list = ['♠A8♣A♦AJ10♥AQ109', '♠KQ1097♣Q10♦9♥K8', '♣KJ987♦KQ87♥7']

In [5]:
bin_array = parse_hand_string(mizere_list[1], 'little')
bin_array

array([161, 132, 132,  37], dtype=uint8)

In [6]:
bin_array = parse_hand_string(hand_list[1], 'big')
bin_array

array([ 64,  23, 138,  66], dtype=uint8)

In [33]:
np.binary_repr(bin_array[0])

'10100001'

In [40]:
np.left_shift(bin_array, 1)

array([66,  8,  8, 74], dtype=uint8)

In [37]:
bin_array

array([161, 132, 132,  37], dtype=uint8)

In [18]:
%%time
deal = PrefDealMatrix.from_strings(hand_list)
mizere_deal = PrefDealMatrix.from_strings(mizere_list)
missing_deal = PrefDealMatrix.from_strings(missing_list)

CPU times: user 469 µs, sys: 573 µs, total: 1.04 ms
Wall time: 914 µs


In [10]:
deal.suit_sizes(), mizere_deal.suit_sizes(), missing_deal.suit_sizes()

(array([8, 8, 6, 8], dtype=uint64),
 array([8, 8, 8, 6], dtype=uint64),
 array([7, 8, 8, 7], dtype=uint64))

In [11]:
deal

♠KQJ9 ♣J ♦Q ♥AJT7 
♠8 ♣AKQT ♦KJ7 ♥K8 
♠AT7 ♣987 ♦AT ♥Q9

In [12]:
mizere_deal

♠98 ♣QJ7 ♦QJ87 ♥8 
♠AQ7 ♣A9 ♦A9 ♥Q97 
♠KJT ♣KT8 ♦KT ♥KJ

In [13]:
missing_deal

♠A8 ♣A ♦AJT ♥AQT9 
♠KQT97 ♣QT ♦9 ♥K8 
♠ ♣KJ987 ♦KQ87 ♥7

In [14]:
deal.permute_suits()

In [15]:
deal

♠Q ♣KQJ9 ♦J ♥AJT7 
♠KJ7 ♣8 ♦AKQT ♥K8 
♠AT ♣AT7 ♦987 ♥Q9

In [16]:
deal.contingency_table()

array([[1, 3, 2],
       [4, 1, 3],
       [1, 4, 3],
       [4, 2, 2]], dtype=uint8)

In [13]:
mizere_deal.contingency_table()

array([[2, 3, 3],
       [3, 2, 3],
       [4, 2, 2],
       [1, 3, 2]], dtype=uint8)

In [14]:
mizere_deal.card_matrix

array([[  6, 161,  88],
       [ 49, 132,  74],
       [ 51, 132,  72],
       [  2,  37,  80]], dtype=uint8)